In [11]:
import re

def text_to_dict(text):
    result = {}
    current_section = None
    lines = text.split('\n')
    for line in lines:
        line = line.strip().replace("：", ":").replace("，", ",").replace("。", ".")
        if line.startswith("发展定位"):
            # 提取发展定位名称（支持中文/英文冒号）
            match = re.match(r"发展定位\d+[:：](.+)", line)
            if match:
                current_section = match.group(1).strip()
                result[current_section] = {"分析": []}  # 初始化一个空列表用于存储分析内容
        elif current_section and line.startswith("- "):
            # 提取分析内容
            analysis_content = line[2:].strip()  # 去掉"- "并去除多余空格
            if analysis_content:
                result[current_section]["分析"].append(analysis_content)
        elif current_section and line.startswith("发展定位"):
            # 如果遇到新的发展定位，但当前section还未处理完，打印警告
            print(f"Warning: New development position found before finishing current section - {line}")
    return result

# 测试文本
text = '''
发展定位1：打造特色农业与生态旅游相结合的综合性旅游目的地
发展定位1的具体分析：- 市场空白：金田村拥有丰富的自然资源和温和湿润的气候，适合发展特色农业和生态旅游。根据市场空白分析，金田村在特色农产品和乡村旅游方面存在巨大的市场空白，具有巨大的发展潜力。
- 成功案例：参考福建省武夷山市的武夷村，成功将茶叶文化和生态旅游结合，吸引了大量游客，实现了农业与旅游的有机结合。
- 大方向作坊：通过技术创新和品种改良，提高农产品的质量和产量，同时开发乡村旅游项目，如农家乐、民宿、休闲观光等。
- 风险分析：自然灾害风险可能导致基础设施损坏，影响经济收入。需要加强农业保险和防灾减灾措施。

发展定位2：建设绿色、有机农产品生产基地
发展定位2的具体分析：- 市场空白：消费者对高品质、绿色、有机农产品的需求逐年增加。市场分析显示，消费者愿意为健康、安全、环保的农产品支付更高的价格。
- 成功案例：参考某丘陵地区特色农业发展成功的案例，如福建省武夷村的茶叶种植，通过品牌建设和市场推广，提升了产品的附加值和市场竞争力。
- 大方向作坊：通过加大绿色、有机农产品的生产力度，满足市场需求，同时提升金田村农产品的知名度和市场竞争力。
- 风险分析：自然灾害风险可能导致农产品产量下降，影响经济收入。需要加强农业保险和防灾减灾措施，确保农业生产的稳定。

发展定位3：培育乡村文化创意产业
发展定位3的具体分析：- 市场空白：乡村文化创意产业市场需求逐年增加，尤其是在城市居民对乡村文化体验需求增长的背景下。
- 成功案例：参考某乡村文化创意产业成功的案例，通过文化活动、手工艺品制作和乡村博物馆等项目，吸引了大量游客。
- 大方向作坊：通过挖掘和保护乡村传统文化，开发文化创意产品，举办文化活动，提升乡村的文化吸引力。
- 风险分析：市场竞争激烈，需要加强品牌建设和市场推广，确保项目的可持续发展。
'''

# 调用函数
formatted_response = text_to_dict(text)
print("Formatted Response:")
print(formatted_response)

Formatted Response:
{'打造特色农业与生态旅游相结合的综合性旅游目的地': {'分析': ['成功案例:参考福建省武夷山市的武夷村,成功将茶叶文化和生态旅游结合,吸引了大量游客,实现了农业与旅游的有机结合.', '大方向作坊:通过技术创新和品种改良,提高农产品的质量和产量,同时开发乡村旅游项目,如农家乐、民宿、休闲观光等.', '风险分析:自然灾害风险可能导致基础设施损坏,影响经济收入.需要加强农业保险和防灾减灾措施.']}, '建设绿色、有机农产品生产基地': {'分析': ['成功案例:参考某丘陵地区特色农业发展成功的案例,如福建省武夷村的茶叶种植,通过品牌建设和市场推广,提升了产品的附加值和市场竞争力.', '大方向作坊:通过加大绿色、有机农产品的生产力度,满足市场需求,同时提升金田村农产品的知名度和市场竞争力.', '风险分析:自然灾害风险可能导致农产品产量下降,影响经济收入.需要加强农业保险和防灾减灾措施,确保农业生产的稳定.']}, '培育乡村文化创意产业': {'分析': ['成功案例:参考某乡村文化创意产业成功的案例,通过文化活动、手工艺品制作和乡村博物馆等项目,吸引了大量游客.', '大方向作坊:通过挖掘和保护乡村传统文化,开发文化创意产品,举办文化活动,提升乡村的文化吸引力.', '风险分析:市场竞争激烈,需要加强品牌建设和市场推广,确保项目的可持续发展.']}}


In [ ]:
import re

def text_to_dict(text):
    result = {}
    current_section = None
    lines = text.split('\n')
    for line in lines:
        line = line.strip().replace("：", ":").replace("，", ",").replace("。", ".")
        if line.startswith("发展定位"):
            # 提取发展定位名称（支持中文/英文冒号）
            match = re.match(r"发展定位\d+[:：](.+)", line)
            if match:
                current_section = match.group(1).strip()
                result[current_section] = {}  # 初始化一个空字典用于存储该定位的内容
        elif current_section:
            # 提取该定位下的内容
            key_value = line.strip()
            # 使用正则表达式提取键值对
            kv_match = re.match(r"^(.*?):\s*(.+)$", key_value)
            if kv_match:
                key = kv_match.group(1).strip()
                value = kv_match.group(2).strip()
                if value.strip() == "":
                    print(f"Warning: Empty value for key '{key}' in line - {line}")
                    continue
                result[current_section][key] = value
            else:
                print(f"Warning: Invalid key-value format - {line}")
    return result

# 测试文本
text = '''
发展定位1：打造特色农业与生态旅游相结合的综合性旅游目的地
发展定位1的具体分析：-市场空白：金田村拥有丰富的自然资源和温和湿润的气候，适合发展特色农业和生态旅游。根据市场空白分析，金田村在特色农产品和乡村旅游方面存在巨大的市场空白，具有巨大的发展潜力。-成功案例：参考福建省武夷山市的武夷村，成功将茶叶文化和生态旅游结合，吸引了大量游客，实现了农业与旅游的有机结合。-大方向作坊：通过技术创新和品种改良，提高农产品的质量和产量，同时开发乡村旅游项目，如农家乐、民宿、休闲观光等。-风险分析：自然灾害风险可能导致基础设施损坏，影响经济收入。需要加强农业保险和防灾减灾措施。

发展定位2：建设绿色、有机农产品生产基地
发展定位2的具体分析：-市场空白：消费者对高品质、绿色、有机农产品的需求逐年增加。市场分析显示，消费者愿意为健康、安全、环保的农产品支付更高的价格。-成功案例：参考某丘陵地区特色农业发展成功的案例，如福建省武夷村的茶叶种植，通过品牌建设和市场推广，提升了产品的附加值和市场竞争力。-大方向作坊：通过加大绿色、有机农产品的生产力度，满足市场需求，同时提升金田村农产品的知名度和市场竞争力。-风险分析：自然灾害风险可能导致农产品产量下降，影响经济收入。需要加强农业保险和防灾减灾措施，确保农业生产的稳定。

发展定位3：培育乡村文化创意产业
发展定位3的具体分析：-市场空白：乡村文化创意产业市场需求逐年增加，尤其是在城市居民对乡村文化体验需求增长的背景下。-成功案例：参考某乡村文化创意产业成功的案例，通过文化活动、手工艺品制作和乡村博物馆等项目，吸引了大量游客。-大方向作坊：通过挖掘和保护乡村传统文化，开发文化创意产品，举办文化活动，提升乡村的文化吸引力。-风险分析：市场竞争激烈，需要加强品牌建设和市场推广，确保项目的可持续发展。
'''

# 调用函数
formatted_response = text_to_dict(text)
print("Formatted Response:")
print(formatted_response)

Formatted Response:
{'打造特色农业与生态旅游相结合的综合性旅游目的地': {'- 成功案例': '参考福建省武夷山市的武夷村,成功将茶叶文化和生态旅游结合,吸引了大量游客,实现了农业与旅游的有机结合.', '- 大方向作坊': '通过技术创新和品种改良,提高农产品的质量和产量,同时开发乡村旅游项目,如农家乐、民宿、休闲观光等.', '- 风险分析': '自然灾害风险可能导致基础设施损坏,影响经济收入.需要加强农业保险和防灾减灾措施.'}, '建设绿色、有机农产品生产基地': {'- 成功案例': '参考某丘陵地区特色农业发展成功的案例,如福建省武夷村的茶叶种植,通过品牌建设和市场推广,提升了产品的附加值和市场竞争力.', '- 大方向作坊': '通过加大绿色、有机农产品的生产力度,满足市场需求,同时提升金田村农产品的知名度和市场竞争力.', '- 风险分析': '自然灾害风险可能导致农产品产量下降,影响经济收入.需要加强农业保险和防灾减灾措施,确保农业生产的稳定.'}, '培育乡村文化创意产业': {'- 成功案例': '参考某乡村文化创意产业成功的案例,通过文化活动、手工艺品制作和乡村博物馆等项目,吸引了大量游客.', '- 大方向作坊': '通过挖掘和保护乡村传统文化,开发文化创意产品,举办文化活动,提升乡村的文化吸引力.', '- 风险分析': '市场竞争激烈,需要加强品牌建设和市场推广,确保项目的可持续发展.'}}


In [14]:
import re

def text_to_dict(text):
    result = {}
    current_section = None
    lines = text.split('\n')
    for line in lines:
        line = line.strip().replace("：", ":").replace("，", ",").replace("。", ".")
        if line.startswith("发展定位"):
            # 提取发展定位名称（支持中文/英文冒号）
            match = re.match(r"发展定位\d+[:：](.+)", line)
            if match:
                current_section = match.group(1).strip()
                result[current_section] = {}  # 初始化一个空字典用于存储该定位的内容
        elif current_section and line.startswith("-"):
            # 提取该定位下的具体分析内容
            # 去掉开头的"-"并去除多余空格
            analysis_content = line[1:].strip()
            # 使用正则表达式提取键值对
            kv_match = re.match(r"^(.*?):\s*(.+)$", analysis_content)
            if kv_match:
                key = kv_match.group(1).strip()
                value = kv_match.group(2).strip()
                if value.strip() == "":
                    print(f"Warning: Empty value for key '{key}' in line - {line}")
                    continue
                result[current_section][key] = value
            else:
                print(f"Warning: Invalid key-value format - {line}")
    return result

# 测试文本
text = '''
发展定位1：打造特色农业与生态旅游相结合的综合性旅游目的地
发展定位1的具体分析：-市场空白：金田村拥有丰富的自然资源和温和湿润的气候，适合发展特色农业和生态旅游。根据市场空白分析，金田村在特色农产品和乡村旅游方面存在巨大的市场空白，具有巨大的发展潜力。-成功案例：参考福建省武夷山市的武夷村，成功将茶叶文化和生态旅游结合，吸引了大量游客，实现了农业与旅游的有机结合。-大方向作坊：通过技术创新和品种改良，提高农产品的质量和产量，同时开发乡村旅游项目，如农家乐、民宿、休闲观光等。-风险分析：自然灾害风险可能导致基础设施损坏，影响经济收入。需要加强农业保险和防灾减灾措施。

发展定位2：建设绿色、有机农产品生产基地
发展定位2的具体分析：-市场空白：消费者对高品质、绿色、有机农产品的需求逐年增加。市场分析显示，消费者愿意为健康、安全、环保的农产品支付更高的价格。-成功案例：参考某丘陵地区特色农业发展成功的案例，如福建省武夷村的茶叶种植，通过品牌建设和市场推广，提升了产品的附加值和市场竞争力。-大方向作坊：通过加大绿色、有机农产品的生产力度，满足市场需求，同时提升金田村农产品的知名度和市场竞争力。-风险分析：自然灾害风险可能导致农产品产量下降，影响经济收入。需要加强农业保险和防灾减灾措施，确保农业生产的稳定。

发展定位3：培育乡村文化创意产业
发展定位3的具体分析：-市场空白：乡村文化创意产业市场需求逐年增加，尤其是在城市居民对乡村文化体验需求增长的背景下。-成功案例：参考某乡村文化创意产业成功的案例，通过文化活动、手工艺品制作和乡村博物馆等项目，吸引了大量游客。-大方向作坊：通过挖掘和保护乡村传统文化，开发文化创意产品，举办文化活动，提升乡村的文化吸引力。-风险分析：市场竞争激烈，需要加强品牌建设和市场推广，确保项目的可持续发展。
'''

# 调用函数
formatted_response = text_to_dict(text)
print("Formatted Response:")
print(formatted_response)

Formatted Response:
{'打造特色农业与生态旅游相结合的综合性旅游目的地': {}, '建设绿色、有机农产品生产基地': {}, '培育乡村文化创意产业': {}}
